## 1. Import Required Libraries

Import modul BioPython, NumPy, Pandas, dan library visualization yang diperlukan.

In [ ]:
# Import library yang diperlukan
from Bio import SeqIO
import Bio.pairwise2 as pairwise2
from Bio.pairwise2 import format_alignment
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from datetime import datetime
import json

print('✓ Semua library berhasil diimport!')
print(f'BioPython version: {Bio.__version__}')

## 2. Load FASTA Sequences

Baca dan parse dua file FASTA untuk mengekstrak informasi sekuens.

In [ ]:
# Tentukan path ke file FASTA (sesuaikan dengan lokasi file Anda)
seq1_file = '../biopython/fasta/sus barbatus.fasta'
seq2_file = '../biopython/fasta/sus scrofa.fasta'

# Baca file FASTA pertama
records1 = list(SeqIO.parse(seq1_file, "fasta"))
seq1_record = records1[0]
seq1_id = seq1_record.id
seq1 = str(seq1_record.seq).upper()

# Baca file FASTA kedua
records2 = list(SeqIO.parse(seq2_file, "fasta"))
seq2_record = records2[0]
seq2_id = seq2_record.id
seq2 = str(seq2_record.seq).upper()

print("="*70)
print("INFORMASI SEKUENS")
print("="*70)
print(f"\nSekuens 1: {seq1_id}")
print(f"  Panjang: {len(seq1)} bp")
print(f"  Preview (100 bp pertama): {seq1[:100]}")
print(f"\nSekuens 2: {seq2_id}")
print(f"  Panjang: {len(seq2)} bp")
print(f"  Preview (100 bp pertama): {seq2[:100]}")
print("\n" + "="*70)

## 3. Set Up Scoring Parameters

Tentukan parameter scoring untuk Needleman-Wunsch algorithm:
- **Match score**: Skor jika dua nukleotida sama (positif)
- **Mismatch penalty**: Skor jika dua nukleotida berbeda (negatif)
- **Gap penalty**: Skor untuk insertion/deletion (negatif)

In [ ]:
# Tentukan parameter scoring
match_score = 2       # Skor untuk match
mismatch_score = -1   # Penalty untuk mismatch
gap_penalty = -2      # Penalty untuk gap (insertion/deletion)

print("="*70)
print("PARAMETER SCORING NEEDLEMAN-WUNSCH ALGORITHM")
print("="*70)
print(f"\nMatch score:      {match_score}")
print(f"Mismatch penalty: {mismatch_score}")
print(f"Gap penalty:      {gap_penalty}")
print("\n" + "="*70)
print("\nKeterangan:")
print("- Match score positif memberikan reward jika nukleotida sama")
print("- Mismatch dan gap penalty negatif mengecilkan skor alignment")
print("- Parameter ini berpengaruh pada kualitas hasil alignment")

## 4. Implement Needleman-Wunsch Algorithm

Implementasi NW algorithm menggunakan `Bio.pairwise2.align.globalms()` untuk global alignment dengan scoring matrix kustom.

In [ ]:
print("\n" + "="*70)
print("MENJALANKAN NEEDLEMAN-WUNSCH ALIGNMENT...")
print("="*70)

# Lakukan NW alignment menggunakan globalms
# globalms = global alignment dengan match/mismatch scoring
alignments = pairwise2.align.globalms(
    seq1, 
    seq2, 
    match_score,      # Match score
    mismatch_score,   # Mismatch score
    gap_penalty,      # Gap open penalty
    gap_penalty       # Gap extension penalty
)

print(f"\nJumlah alignment optimal ditemukan: {len(alignments)}")
print(f"Skor alignment tertinggi: {alignments[0][2]}")
print("\n" + "="*70)

## 5. Perform Pairwise Alignment

Analisis hasil alignment dan ekstrak informasi penting.

In [ ]:
# Ambil alignment dengan skor tertinggi
best_alignment = alignments[0]
aligned_seq1, aligned_seq2, score, begin, end = best_alignment

print("\n" + "="*70)
print("HASIL ALIGNMENT NEEDLEMAN-WUNSCH")
print("="*70)
print(f"\nAlignment Score: {score}")
print(f"Panjang alignment: {len(aligned_seq1)} bp")
print(f"Position range: {begin} - {end}")

## 6. Display Alignment Results

Tampilkan hasil alignment dalam format yang rapi dengan match/mismatch indicators.

In [ ]:
# Fungsi untuk menampilkan alignment dengan format yang rapi
def display_alignment(seq1, seq2, line_width=60):
    """
    Tampilkan alignment dengan match/mismatch indicators
    | = match
    . = mismatch
    (space) = gap
    """
    print("\n" + "="*100)
    print("PAIRWISE ALIGNMENT")
    print("="*100 + "\n")
    
    for i in range(0, len(seq1), line_width):
        seq1_block = seq1[i:i+line_width]
        seq2_block = seq2[i:i+line_width]
        
        # Buat matching indicator string
        match_block = ""
        for s1, s2 in zip(seq1_block, seq2_block):
            if s1 == s2:
                match_block += "|"
            elif s1 == "-" or s2 == "-":
                match_block += " "
            else:
                match_block += "."
        
        print(f"Seq1: {seq1_block}")
        print(f"      {match_block}")
        print(f"Seq2: {seq2_block}")
        print()
    print("\n" + "="*100)
    print("Keterangan: | = match, . = mismatch, (space) = gap")
    print("="*100)

# Tampilkan alignment
display_alignment(aligned_seq1, aligned_seq2)

## 7. Analyze Alignment Output

Hitung dan analisis metrik alignment termasuk identity, similarity, dan gap statistics.

In [ ]:
# Hitung statistik alignment
matches = sum(1 for i, j in zip(aligned_seq1, aligned_seq2) if i == j)
mismatches = sum(1 for i, j in zip(aligned_seq1, aligned_seq2) if i != j and i != '-' and j != '-')
gaps_seq1 = aligned_seq1.count('-')
gaps_seq2 = aligned_seq2.count('-')
total_gaps = gaps_seq1 + gaps_seq2

alignment_length = len(aligned_seq1)

# Hitung persentase
identity = (matches / alignment_length) * 100 if alignment_length > 0 else 0
similarity = ((alignment_length - total_gaps) / alignment_length) * 100 if alignment_length > 0 else 0
gap_percentage = (total_gaps / (alignment_length * 2)) * 100 if alignment_length > 0 else 0

# Buat DataFrame untuk statistik
stats_data = {
    'Metrik': [
        'Panjang Alignment',
        'Matches',
        'Mismatches',
        'Gap Seq1',
        'Gap Seq2',
        'Total Gaps',
        'Identity (%)',
        'Similarity (%)',
        'Gap Percentage (%)',
        'Alignment Score'
    ],
    'Nilai': [
        alignment_length,
        matches,
        mismatches,
        gaps_seq1,
        gaps_seq2,
        total_gaps,
        f'{identity:.2f}',
        f'{similarity:.2f}',
        f'{gap_percentage:.2f}',
        score
    ]
}

stats_df = pd.DataFrame(stats_data)

print("\n" + "="*70)
print("STATISTIK ALIGNMENT")
print("="*70)
print(stats_df.to_string(index=False))
print("\n" + "="*70)

### Interpretasi Hasil Statistik

In [ ]:
print("\n" + "="*70)
print("INTERPRETASI HASIL ALIGNMENT")
print("="*70)

print(f"\n1. IDENTITY ({identity:.2f}%)")
print(f"   - Persentase nukleotida yang sama di antara kedua sekuens")
print(f"   - Matches: {matches} dari {alignment_length} bp")
if identity > 90:
    print(f"   ➜ Interpretasi: Sekuens SANGAT MIRIP (>90%), kemungkinan spesies yang erat kaitannya")
elif identity > 80:
    print(f"   ➜ Interpretasi: Sekuens MIRIP (80-90%), species yang berdekatan")
elif identity > 70:
    print(f"   ➜ Interpretasi: Sekuens CUKUP MIRIP (70-80%), ada kesamaan signifikan")
else:
    print(f"   ➜ Interpretasi: Sekuens BERBEDA (<70%), kemungkinan evolusi jauh")

print(f"\n2. MISMATCHES ({mismatches})")
print(f"   - Jumlah posisi di mana nukleotida berbeda")
print(f"   - Mismatch rate: {(mismatches/alignment_length)*100:.2f}%")

print(f"\n3. GAPS ({total_gaps})")
print(f"   - Gaps di Seq1: {gaps_seq1}")
print(f"   - Gaps di Seq2: {gaps_seq2}")
print(f"   - Gap percentage: {gap_percentage:.2f}%")
if gap_percentage < 5:
    print(f"   ➜ Interpretasi: Sangat sedikit indel (insertion/deletion), alignment stabil")
elif gap_percentage < 10:
    print(f"   ➜ Interpretasi: Moderate indel, ada beberapa perbedaan panjang")
else:
    print(f"   ➜ Interpretasi: Banyak indel, signifikan structural differences")

print(f"\n4. ALIGNMENT SCORE ({score})")
print(f"   - Skor gabungan dari matches, mismatches, dan gaps")
print(f"   - Semakin tinggi skor, semakin baik alignment")

print("\n" + "="*70)

## 8. Visualisasi Hasil Alignment

In [ ]:
# Buat visualisasi
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Needleman-Wunsch Alignment Analysis', fontsize=16, fontweight='bold')

# 1. Identity Pie Chart
ax1 = axes[0, 0]
identity_data = [matches, mismatches]
colors = ['#2ecc71', '#e74c3c']
ax1.pie(identity_data, labels=['Matches', 'Mismatches'], autopct='%1.1f%%', 
        colors=colors, startangle=90)
ax1.set_title('Identity Analysis', fontweight='bold')

# 2. Sequence Composition
ax2 = axes[0, 1]
categories = ['Matches', 'Mismatches', 'Gaps']
values = [matches, mismatches, total_gaps]
bars = ax2.bar(categories, values, color=['#2ecc71', '#e74c3c', '#f39c12'])
ax2.set_ylabel('Count')
ax2.set_title('Alignment Composition', fontweight='bold')
for bar in bars:
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height,
            f'{int(height)}', ha='center', va='bottom')

# 3. Gap Distribution
ax3 = axes[1, 0]
gap_data = [gaps_seq1, gaps_seq2]
ax3.bar(['Seq1 Gaps', 'Seq2 Gaps'], gap_data, color=['#3498db', '#9b59b6'])
ax3.set_ylabel('Count')
ax3.set_title('Gap Distribution', fontweight='bold')
for i, v in enumerate(gap_data):
    ax3.text(i, v, str(v), ha='center', va='bottom')

# 4. Summary Statistics
ax4 = axes[1, 1]
ax4.axis('off')
summary_text = f"""
ALIGNMENT SUMMARY

Alignment Length: {alignment_length} bp
Identity: {identity:.2f}%
Alignment Score: {score}

Matches: {matches} ({(matches/alignment_length)*100:.2f}%)
Mismatches: {mismatches} ({(mismatches/alignment_length)*100:.2f}%)
Total Gaps: {total_gaps} ({gap_percentage:.2f}%)

Seq1 Length: {len(seq1)} bp
Seq2 Length: {len(seq2)} bp
"""
ax4.text(0.1, 0.5, summary_text, fontsize=11, verticalalignment='center',
        fontfamily='monospace', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.savefig('../output/alignment_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Visualisasi disimpan ke: ../output/alignment_analysis.png")

## 9. Simpan Hasil ke File

In [ ]:
# Simpan hasil ke file text
output_file = Path('../output/nw_alignment_result.txt')
output_file.parent.mkdir(exist_ok=True)

with open(output_file, 'w') as f:
    f.write("="*80 + "\n")
    f.write("HASIL NEEDLEMAN-WUNSCH ALIGNMENT\n")
    f.write("="*80 + "\n\n")
    
    f.write(f"Tanggal: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
    
    f.write("INFORMASI SEKUENS\n")
    f.write("-"*80 + "\n")
    f.write(f"Sekuens 1: {seq1_id}\n")
    f.write(f"  Panjang: {len(seq1)} bp\n")
    f.write(f"  Preview: {seq1[:100]}...\n\n")
    
    f.write(f"Sekuens 2: {seq2_id}\n")
    f.write(f"  Panjang: {len(seq2)} bp\n")
    f.write(f"  Preview: {seq2[:100]}...\n\n")
    
    f.write("HASIL ALIGNMENT\n")
    f.write("-"*80 + "\n")
    f.write(f"Skor: {score}\n\n")
    
    # Tulis alignment
    line_width = 50
    for i in range(0, len(aligned_seq1), line_width):
        seq1_block = aligned_seq1[i:i+line_width]
        seq2_block = aligned_seq2[i:i+line_width]
        
        match_block = ""
        for s1, s2 in zip(seq1_block, seq2_block):
            if s1 == s2:
                match_block += "|"
            elif s1 == "-" or s2 == "-":
                match_block += " "
            else:
                match_block += "."
        
        f.write(f"Seq1: {seq1_block}\n")
        f.write(f"      {match_block}\n")
        f.write(f"Seq2: {seq2_block}\n\n")
    
    f.write("STATISTIK ALIGNMENT\n")
    f.write("-"*80 + "\n")
    f.write(f"Panjang alignment: {alignment_length} bp\n")
    f.write(f"Matches: {matches} ({(matches/alignment_length)*100:.2f}%)\n")
    f.write(f"Mismatches: {mismatches}\n")
    f.write(f"Gaps total: {total_gaps}\n")
    f.write(f"Identity: {identity:.2f}%\n")
    f.write(f"Gap percentage: {gap_percentage:.2f}%\n")

print(f"✓ Hasil disimpan ke: {output_file}")

In [ ]:
# Simpan hasil ke JSON untuk data analysis lebih lanjut
import json

result_json = Path('../output/nw_alignment_result.json')

result_data = {
    'metadata': {
        'timestamp': datetime.now().isoformat(),
        'algorithm': 'Needleman-Wunsch (Global Alignment)'
    },
    'sequences': {
        'seq1': {'id': seq1_id, 'length': len(seq1)},
        'seq2': {'id': seq2_id, 'length': len(seq2)}
    },
    'alignment': {
        'aligned_seq1': aligned_seq1,
        'aligned_seq2': aligned_seq2,
        'score': score
    },
    'statistics': {
        'alignment_length': alignment_length,
        'matches': matches,
        'mismatches': mismatches,
        'gaps_seq1': gaps_seq1,
        'gaps_seq2': gaps_seq2,
        'total_gaps': total_gaps,
        'identity_percent': round(identity, 2),
        'gap_percentage': round(gap_percentage, 2)
    }
}

with open(result_json, 'w') as f:
    json.dump(result_data, f, indent=2)

print(f"✓ Hasil JSON disimpan ke: {result_json}")

## 10. Kesimpulan dan Analisis Biologis

In [ ]:
print("\n" + "="*70)
print("KESIMPULAN DAN ANALISIS BIOLOGIS")
print("="*70)

print(f"\n📊 RINGKASAN HASIL:")
print(f"   Dua sekuens {seq1_id} dan {seq2_id} telah dianalisis")
print(f"   menggunakan Needleman-Wunsch Algorithm.")

print(f"\n🔬 TEMUAN UTAMA:")
print(f"   1. Identity: {identity:.2f}%")
print(f"      - Menunjukkan tingkat kesamaan antar kedua sekuens")

print(f"\n   2. Alignment Score: {score}")
print(f"      - Metrik kualitas global dari alignment")

print(f"\n   3. Gap Analysis: {gap_percentage:.2f}%")
print(f"      - Menunjukkan adanya insertion/deletion events")
print(f"      - Indikator evolutionary distance")

print(f"\n💡 INTERPRETASI BIOLOGIS:")
if identity > 95:
    print(f"   - Sekuens NEARLY IDENTICAL, kemungkinan same species atau very closely related")
    print(f"   - Evolusi yang sangat recent atau minimal divergence")
elif identity > 85:
    print(f"   - Sekuens HIGHLY SIMILAR, kemungkinan same genus atau close species")
    print(f"   - Common ancestor dari waktu yang relatif recent")
elif identity > 75:
    print(f"   - Sekuens MODERATELY SIMILAR, different species tapi related genus")
    print(f"   - Evolutionary conservation dari regions penting")
else:
    print(f"   - Sekuens DISTANTLY RELATED, significant evolutionary divergence")
    print(f"   - Hanya highly conserved regions yang maintain similarity")

print(f"\n📈 NEXT STEPS:")
print(f"   1. Cek conserved regions yang penting secara fungsional")
print(f"   2. Identifikasi indel patterns untuk evolutionary analysis")
print(f"   3. Compare dengan alignment tools lain (BLAST, MUSCLE)")
print(f"   4. Analisis phylogenetic untuk tree construction")

print("\n" + "="*70)